<a href="https://colab.research.google.com/github/liupengzhouyi/LearningColaboratory/blob/master/%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手写数字识别

## 导入模块

In [0]:
import torch
import torchvision

## 查看版本

In [2]:
print('torch.version:', torch.__version__)
print('torchvision.version:', torchvision.__version__)

torch.version: 1.4.0
torchvision.version: 0.5.0


## 下载训练数据

In [5]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw



Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


### 处理训练数据

> (50 * 1 * 28 *28)

In [0]:
trainDateSet = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

## 下载测试数据

In [0]:
test_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=False
)

### 处理测试数据

In [8]:
test_x = torch.unsqueeze(input=test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.
test_y = test_data.test_labels[:2000]

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## 建立CNN模型

> 输入 (1  * 28 * 28)

>   👇

> conv-1: (16 * 28 * 28)

>   👇

> pool-1: (16 * 14 * 14)

>   👇

> oncv-2: (32 * 14 * 14)

>   👇

> pool-2: (32 * 7 * 7)

>   👇

> linear: (32 * 7 * 7)

>   👇

> output: (10)

In [0]:
class LpModel(torch.nn.Module):

    def __init__(self):
        super(LpModel, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, 
                            kernel_size=5, stride=1, padding=2,),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16, out_channels=32, 
                            kernel_size=5, stride=1, padding=2,),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        self.out = torch.nn.Linear(32 * 7 * 7, 10)
    
    def forward(self, input):
        input = self.conv1(input)
        input = self.conv2(input)
        input = input.view(input.size(0), -1)
        output = self.out(input)
        return output

## 实例化模型

In [0]:
model = LpModel()

## 输出模型的结构

In [11]:
print(model)

LpModel(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


## 创建优化器

In [0]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

## 创建损失函数

In [0]:
loss_func = torch.nn.CrossEntropyLoss()

## 训练

In [14]:
for epoch in range(2):
    for step, (data, label) in enumerate(trainDateSet):
        data = torch.autograd.Variable(data)
        output = model(data)
        loss = loss_func(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print('times:', step)

times: 0
times: 100
times: 200
times: 300
times: 400
times: 500
times: 600
times: 700
times: 800
times: 900
times: 1000
times: 1100
times: 0
times: 100
times: 200
times: 300
times: 400
times: 500
times: 600
times: 700
times: 800
times: 900
times: 1000
times: 1100


## 计算正确率

$$ \frac{rightNumber}{testNumber} $$

## 测试

In [25]:
test_output = model(test_x)

tensor([ -4.1110,  -8.3949,  -4.2038,   1.2952, -13.6527,  -7.3260, -27.1470,
         12.1747,  -8.8163,   1.9086], grad_fn=<SelectBackward>)


## 获取测试结果

In [0]:
prey_y = torch.max(test_output, 1)[1].data.numpy()

## 统计测试结果

In [27]:
rightNumber = float((prey_y == test_y.data.numpy()).astype(int).sum())
print(rightNumber)
testNumber = float(len(test_y))
print(testNumber)

1947.0
2000.0


## 计算正确率

In [28]:
accuracy = rightNumber / testNumber
print(accuracy)

0.9735


## 查看出错点

In [29]:
for i in range(2000):
    if (prey_y[i] != test_y[i]):
        print(test_y[i], prey_y[i])

tensor(5) 6
tensor(3) 5
tensor(7) 9
tensor(9) 4
tensor(7) 2
tensor(4) 6
tensor(4) 9
tensor(8) 5
tensor(9) 8
tensor(8) 0
tensor(8) 2
tensor(7) 3
tensor(4) 6
tensor(2) 3
tensor(7) 2
tensor(0) 5
tensor(7) 5
tensor(4) 9
tensor(6) 5
tensor(9) 8
tensor(4) 3
tensor(7) 3
tensor(3) 5
tensor(8) 9
tensor(6) 0
tensor(6) 5
tensor(8) 1
tensor(7) 3
tensor(4) 6
tensor(7) 9
tensor(6) 8
tensor(7) 2
tensor(7) 2
tensor(9) 5
tensor(7) 1
tensor(3) 5
tensor(7) 2
tensor(5) 6
tensor(5) 3
tensor(7) 3
tensor(7) 9
tensor(4) 6
tensor(3) 7
tensor(9) 5
tensor(7) 1
tensor(7) 9
tensor(7) 2
tensor(8) 3
tensor(9) 8
tensor(7) 2
tensor(8) 2
tensor(4) 9
tensor(4) 1
